In [117]:
from typing import Iterable, List, Dict, Callable
from collections import defaultdict
from operator import itemgetter
import csv
import math

In [104]:
# cargos - list of all cargos
# container_size - size of containers to fit in cargos
# next_avail_contrainer - function that returns one of existing containers or None,
# if it's impossible to fit in already existing containers
# Number of comparisons is
def pack(cargos: Iterable[int], container_size: int, next_avail_contrainer: Callable, idxs=False) -> defaultdict:
    containers = defaultdict(list)
    containers_weight = defaultdict(int)
    container_n = 1
    containers_weight[container_n] = 0
    containers_idx = defaultdict(list)

    for n, c in enumerate(cargos):
        to_append = container_n
        if containers_weight[to_append] + c > container_size:
            to_append = next_avail_contrainer(c, containers_weight, container_size)
            if to_append is None:
                # can't append cargo to existing container, so add new
                container_n += 1
                containers_weight[container_n] = c
                containers[container_n].append(c)
                containers_idx[container_n].append(n)          
                continue
        containers_weight[to_append] += c
        containers[to_append].append(c)
        containers_idx[container_n].append(n)
    if idxs:
        return containers_idx
    return containers

In [150]:
def csv_writer(data, path):
    """
    Write data to a CSV file path
    """
    with open(path, "w", newline='') as csv_file:
        writer = csv.writer(csv_file, delimiter=',')
        for line in data:
            writer.writerow(line)

#for further import to google sheets
def pack_to_csv(cargos: Iterable[int], container_size: int, next_avail_contrainer: Callable, table_name):
    data = [
        [table_name],
        ["Container", "Cargo"],
        [""] + list(range(1, len(cargos)+1))
    ]
    res = pack(cargos, container_size, next_avail_contrainer, idxs=True)
    empty = [""] * len(cargos)
    for n, cargos_idxs in res.items():
        csv_row = list(empty)
        for ci in cargos_idxs:
            csv_row[ci] = cargos[ci]
        data.append([n] + csv_row)
    return data
    # csv_writer(data, filename)

In [57]:
# O(N)
def NFA(current_weight: int, containers_weight: dict, container_size: int) -> int:
    # pick last container
    k = list(containers_weight.keys())[-1]
    if containers_weight[k] + current_weight <= container_size:
        return k
    return None

In [57]:
def NFA(cargos: Iterable[int], container_size: int) -> Dict:
    containers = defaultdict(list)
    container_n = 1
    cur_weight = 0
    for c in cargos:
        if cur_weight + c > container_size:
            cur_weight = c
            container_n += 1
        else:
            cur_weight += c
        containers[container_n].append(c)
    return containers

In [60]:
# helper function to find and fill avail containers
# O(N^2)
def FFA(current_weight: int, containers_weight: dict, container_size: int) -> int:
    for k, v in containers_weight.items():
        if v + current_weight <= container_size:
            return k
    return None

In [61]:
# helper function to get min weight container
# O(N^2)
def WFA(current_weight: int, containers_weight: dict, container_size: int) -> int:
    k = min(containers_weight.items(), key=itemgetter(1))[0]
    if containers_weight[k] + current_weight <= container_size:
        return k
    return None

In [62]:
# find max avail weight
# O(N^2)
def BFA(current_weight: int, containers_weight: dict, container_size: int) -> int:
    # max_weight = 0
    max_weight_k = None
    for k, v in containers_weight.items():
        if v + current_weight <= container_size:
            if max_weight_k is None or containers_weight[max_weight_k] < v:
                max_weight_k = k
    return max_weight_k

In [153]:
methods = {
    "NFA": NFA,
    "FFA": FFA,
    "WFA": WFA,
    "BFA": BFA
}

container_size = 100   

def run_all(cargos: List, prefix):
    print(f"Cargos: {cargos}")
    print(f"Min possible: {math.ceil(sum(cargos)/container_size)}")
    cargos_comb = {
        "not sorted": cargos,
        "sorted": sorted(cargos, reverse=True)
    }
    result_dict = {}
    result_str = ""
    result_data = []
    for name, func in methods.items():
        for cargo_name, cargo_items in cargos_comb.items():
            full_name = f"{name} - {cargo_name}"
            
            result_data += pack_to_csv(cargo_items, container_size, func, f"{prefix}-{full_name}")
            result = pack(cargo_items, container_size, func)
            
            n_of_containers = len(result)
            result_dict[full_name] = n_of_containers
            result_str += f"==== {full_name} - {n_of_containers}:\n"
            for k, v in result.items():
                result_str += f"\t{k}: {v}\n"
    best_result = min(result_dict.values())
    print(f"Best is: {best_result} : {[k for k, v in result_dict.items() if v== best_result]}")
    print(result_str)
    csv_writer(result_data, f"./{prefix}-resuts.csv")

In [154]:
# example 
cargos = [int(c) for c in "23 3 51 20 51 42 52 12 4 7 59 58 25 94 18".split()]
run_all(cargos, "example")

Cargos: [23, 3, 51, 20, 51, 42, 52, 12, 4, 7, 59, 58, 25, 94, 18]
Min possible: 6
Best is: 6 : ['FFA - not sorted', 'FFA - sorted', 'WFA - not sorted', 'WFA - sorted', 'BFA - not sorted', 'BFA - sorted']
==== NFA - not sorted - 7:
	1: [23, 3, 51, 20]
	2: [51, 42]
	3: [52, 12, 4, 7]
	4: [59]
	5: [58, 25]
	6: [94]
	7: [18]
==== NFA - sorted - 8:
	1: [94]
	2: [59]
	3: [58]
	4: [52]
	5: [51]
	6: [51, 42]
	7: [25, 23, 20, 18, 12]
	8: [7, 4, 3]
==== FFA - not sorted - 6:
	1: [23, 3, 51, 20]
	2: [51, 42]
	3: [52, 12, 4, 7, 18]
	4: [59]
	5: [58, 25]
	6: [94]
==== FFA - sorted - 6:
	1: [94, 4]
	2: [59, 25, 12, 3]
	3: [58, 23, 18]
	4: [52, 20]
	5: [51]
	6: [51, 42, 7]
==== WFA - not sorted - 6:
	1: [23, 3, 51, 20]
	2: [51, 42]
	3: [52, 12, 4, 7]
	4: [59, 18]
	5: [58, 25]
	6: [94]
==== WFA - sorted - 6:
	1: [94]
	2: [59, 18, 3]
	3: [58, 20]
	4: [52, 23, 12]
	5: [51, 25, 4]
	6: [51, 42, 7]
==== BFA - not sorted - 6:
	1: [23, 3, 51, 20]
	2: [51, 42]
	3: [52, 12, 4, 7, 18]
	4: [59]
	5: [58, 25]
	6: 

In [155]:
# Row 1
cargos = [53, 66, 97, 48, 47, 72, 16, 86, 12, 59, 77, 51, 53, 97, 32, 3, 35, 51, 7, 98]
run_all(cargos, "row_1")

Cargos: [53, 66, 97, 48, 47, 72, 16, 86, 12, 59, 77, 51, 53, 97, 32, 3, 35, 51, 7, 98]
Min possible: 11
Best is: 12 : ['FFA - sorted', 'WFA - sorted', 'BFA - sorted']
==== NFA - not sorted - 14:
	1: [53]
	2: [66]
	3: [97]
	4: [48, 47]
	5: [72, 16]
	6: [86, 12]
	7: [59]
	8: [77]
	9: [51]
	10: [53]
	11: [97]
	12: [32, 3, 35]
	13: [51, 7]
	14: [98]
==== NFA - sorted - 14:
	1: [98]
	2: [97]
	3: [97]
	4: [86]
	5: [77]
	6: [72]
	7: [66]
	8: [59]
	9: [53]
	10: [53]
	11: [51]
	12: [51, 48]
	13: [47, 35]
	14: [32, 16, 12, 7, 3]
==== FFA - not sorted - 13:
	1: [53, 32]
	2: [66]
	3: [97]
	4: [48, 47]
	5: [72, 16]
	6: [86, 12]
	7: [59, 35]
	8: [77]
	9: [51]
	10: [53]
	11: [97, 3]
	12: [51, 7]
	13: [98]
==== FFA - sorted - 12:
	1: [98]
	2: [97, 3]
	3: [97]
	4: [86, 12]
	5: [77, 16, 7]
	6: [72]
	7: [66, 32]
	8: [59, 35]
	9: [53, 47]
	10: [53]
	11: [51]
	12: [51, 48]
==== WFA - not sorted - 13:
	1: [53, 35]
	2: [66]
	3: [97]
	4: [48, 47]
	5: [72, 16]
	6: [86, 12]
	7: [59]
	8: [77]
	9: [51, 32]
	10: [

In [156]:
# Row 2
cargos = [49, 54, 61, 6, 34, 3, 25, 84, 28, 97, 63, 33, 15, 60, 81, 14, 85, 97, 51, 97]
run_all(cargos, "row_2")

Cargos: [49, 54, 61, 6, 34, 3, 25, 84, 28, 97, 63, 33, 15, 60, 81, 14, 85, 97, 51, 97]
Min possible: 11
Best is: 11 : ['FFA - sorted', 'WFA - sorted', 'BFA - sorted']
==== NFA - not sorted - 14:
	1: [49]
	2: [54]
	3: [61, 6]
	4: [34, 3, 25]
	5: [84]
	6: [28]
	7: [97]
	8: [63, 33]
	9: [15, 60]
	10: [81, 14]
	11: [85]
	12: [97]
	13: [51]
	14: [97]
==== NFA - sorted - 13:
	1: [97]
	2: [97]
	3: [97]
	4: [85]
	5: [84]
	6: [81]
	7: [63]
	8: [61]
	9: [60]
	10: [54]
	11: [51, 49]
	12: [34, 33, 28]
	13: [25, 15, 14, 6, 3]
==== FFA - not sorted - 12:
	1: [49, 34, 15]
	2: [54, 28]
	3: [61, 6, 3, 25]
	4: [84]
	5: [97]
	6: [63, 33]
	7: [60]
	8: [81, 14]
	9: [85]
	10: [97]
	11: [51]
	12: [97]
==== FFA - sorted - 11:
	1: [97, 3]
	2: [97]
	3: [97]
	4: [85, 15]
	5: [84, 14]
	6: [81, 6]
	7: [63, 34]
	8: [61, 33]
	9: [60, 28]
	10: [54, 25]
	11: [51, 49]
==== WFA - not sorted - 12:
	1: [49, 34]
	2: [54, 28, 15]
	3: [61, 6, 3, 25]
	4: [84]
	5: [97]
	6: [63, 33]
	7: [60]
	8: [81, 14]
	9: [85]
	10: [97]
	11:

In [157]:
# Row 3
cargos = [8, 29, 49, 13, 79, 34, 16, 14, 85, 75, 65, 86, 30, 26, 92, 16, 29, 69, 52, 9]
run_all(cargos, "row_3")

Cargos: [8, 29, 49, 13, 79, 34, 16, 14, 85, 75, 65, 86, 30, 26, 92, 16, 29, 69, 52, 9]
Min possible: 9
Best is: 10 : ['FFA - not sorted', 'FFA - sorted', 'WFA - not sorted', 'WFA - sorted', 'BFA - not sorted', 'BFA - sorted']
==== NFA - not sorted - 12:
	1: [8, 29, 49, 13]
	2: [79]
	3: [34, 16, 14]
	4: [85]
	5: [75]
	6: [65]
	7: [86]
	8: [30, 26]
	9: [92]
	10: [16, 29]
	11: [69]
	12: [52, 9]
==== NFA - sorted - 12:
	1: [92]
	2: [86]
	3: [85]
	4: [79]
	5: [75]
	6: [69]
	7: [65]
	8: [52]
	9: [49, 34]
	10: [30, 29, 29]
	11: [26, 16, 16, 14, 13, 9]
	12: [8]
==== FFA - not sorted - 10:
	1: [8, 29, 49, 13]
	2: [79, 16]
	3: [34, 16, 14, 30]
	4: [85]
	5: [75]
	6: [65, 26]
	7: [86]
	8: [92]
	9: [29, 69]
	10: [52, 9]
==== FFA - sorted - 10:
	1: [92, 8]
	2: [86]
	3: [85]
	4: [79]
	5: [75]
	6: [69, 30]
	7: [65, 29]
	8: [52, 29]
	9: [49, 34]
	10: [26, 16, 16, 14, 13, 9]
==== WFA - not sorted - 10:
	1: [8, 29, 49, 13]
	2: [79]
	3: [34, 16, 14, 30]
	4: [85]
	5: [75, 16]
	6: [65, 26]
	7: [86]
	8: [92]

In [158]:
# Row 1-3
cargos = [53, 66, 97, 48, 47, 72, 16, 86, 12, 59, 77, 51, 53, 97, 32, 3, 35, 51, 7, 98, 
          49, 54, 61, 6, 34, 3, 25, 84, 28, 97, 63, 33, 15, 60, 81, 14, 85, 97, 51, 97,
          8, 29, 49, 13, 79, 34, 16, 14, 85, 75, 65, 86, 30, 26, 92, 16, 29, 69, 52, 9]
run_all(cargos, "row_1_3")

Cargos: [53, 66, 97, 48, 47, 72, 16, 86, 12, 59, 77, 51, 53, 97, 32, 3, 35, 51, 7, 98, 49, 54, 61, 6, 34, 3, 25, 84, 28, 97, 63, 33, 15, 60, 81, 14, 85, 97, 51, 97, 8, 29, 49, 13, 79, 34, 16, 14, 85, 75, 65, 86, 30, 26, 92, 16, 29, 69, 52, 9]
Min possible: 30
Best is: 31 : ['FFA - sorted', 'WFA - sorted', 'BFA - sorted']
==== NFA - not sorted - 40:
	1: [53]
	2: [66]
	3: [97]
	4: [48, 47]
	5: [72, 16]
	6: [86, 12]
	7: [59]
	8: [77]
	9: [51]
	10: [53]
	11: [97]
	12: [32, 3, 35]
	13: [51, 7]
	14: [98]
	15: [49]
	16: [54]
	17: [61, 6]
	18: [34, 3, 25]
	19: [84]
	20: [28]
	21: [97]
	22: [63, 33]
	23: [15, 60]
	24: [81, 14]
	25: [85]
	26: [97]
	27: [51]
	28: [97]
	29: [8, 29, 49, 13]
	30: [79]
	31: [34, 16, 14]
	32: [85]
	33: [75]
	34: [65]
	35: [86]
	36: [30, 26]
	37: [92]
	38: [16, 29]
	39: [69]
	40: [52, 9]
==== NFA - sorted - 39:
	1: [98]
	2: [97]
	3: [97]
	4: [97]
	5: [97]
	6: [97]
	7: [92]
	8: [86]
	9: [86]
	10: [85]
	11: [85]
	12: [84]
	13: [81]
	14: [79]
	15: [77]
	16: [75]
	17: [72]